<a href="https://colab.research.google.com/github/ry007/Alert/blob/master/Importing_a_Graph_into_modelzoo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using custom `tensorflow` models with `lucid`

Getting a tensorflow model to work with lucid's `modelzoo` is a two step process:

* Obtain a "frozen" **graph definition** of the model
* Specify **metadata** of the model in a `lucid/modelzoo` class.

The "frozen" **graph definition** allows us to load the entire graph, *including the learned weights* from a single file.
(Tensorflow usually saves the *structure* of a model's graph separate from the *weights*.)

The **metadata** specifies for example the input range of a model, and which nodes in the graph are interesting to visualize.

## "Freezing" the model's graph

You will need both a **graph definition** file and a **checkpoint** containing the values of the weights.



### Getting the graph definition file

#### If you have a `slim` model, such as from the `tensorflow/models` repository

In this case you can use a tool built into slim to export that model's graph into a graph definition file:

(Here I install Subversion/`svn`, because I don't want to checkout the entire `tensorflow/models` repository. 
Github serves repositories via Subversion as well, and I'm just interested in the `research/slim` subfolder. Using `git` I'd have to clone the entire repository.)

In [1]:
%cd
  
!git clone --quiet https://github.com/tensorflow/models.git

!apt-get install -qq protobuf-compiler python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib PyDrive

!pip install -q pycocotools

%cd ~/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/root/models/research/:/root/models/research/slim/'

!python object_detection/builders/model_builder_test.py

/root
     |████████████████████████████████| 993kB 5.2MB/s 
/root/models/research

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

............s...
----------------------------------------------------------------------
Ran 16 tests in 0.100s

OK (skipped=1)


In [0]:
!apt-get -qq install subversion
!svn checkout https://github.com/tensorflow/models/trunk/research/object_detection

Check the download succeeded:

In [3]:
!ls object_detection

anchor_generators		     legacy
box_coders			     matchers
builders			     meta_architectures
CONTRIBUTING.md			     metrics
core				     model_hparams.py
data				     model_lib.py
data_decoders			     model_lib_test.py
dataset_tools			     model_main.py
dockerfiles			     models
eval_util.py			     model_tpu_main.py
eval_util_test.py		     object_detection_tutorial.ipynb
exporter.py			     predictors
exporter_test.py		     protos
export_inference_graph.py	     __pycache__
export_tflite_ssd_graph_lib.py	     README.md
export_tflite_ssd_graph_lib_test.py  samples
export_tflite_ssd_graph.py	     test_ckpt
g3doc				     test_data
inference			     test_images
__init__.py			     tpu_exporters
inputs.py			     utils
inputs_test.py


And use the `export_inference_graph.py` script to export a GraphDef for the model we want.

You can find the `model_name` for the model you are interested in in the [nets_factory.py](https://github.com/tensorflow/models/blob/master/research/slim/nets/nets_factory.py) file. For us, it's unsurprisingly called '`nasnet_mobile`'.

In [108]:
from google.colab import files
upload=files.upload()
for name, data in upload.items():
  with open(name, 'wb') as f:
    f.write(data)
    print('saved file',name)

Saving frozen_inference_graph.png to frozen_inference_graph.png
saved file frozen_inference_graph.png


In [132]:
!ls

a3c_blogpost			    lstm_object_detection
adversarial_crypto		    marco
adversarial_logit_pairing	    maskgan
adversarial_text		    minigo
adv_imagenet_models		    model.ckpt
astronet			    model.ckpt.data-00000-of-00001
attention_ocr			    model.ckpt.index
audioset			    model.ckpt.meta
autoaugment			    morph_net
autoencoder			    namignizer
brain_coder			    neural_gpu
checkpoint			    neural_programmer
cognitive_mapping_and_planning	    next_frame_prediction
cognitive_planning		    nst_blogpost
compression			    object_detection
cvt_text			    pcl_rl
deep_contextual_bandits		    pipeline.config
deeplab				    ptn
deep_speech			    qa_kg
delf				    README.md
differential_privacy		    real_nvp
domain_adaptation		    rebar
efficient-hrl			    research
feelvos				    resnet
fivo				    saved_model
frozen_inference_graph.pb	    sentiment_analysis
frozen_inference_graph.pb.modelzoo  seq2species
frozen_inference_graph.png	    setup.py
gan				    skip_thoughts
global_objectives		    slim
h

In [87]:
%cd ..

/root/models/research


In [74]:
import os
cwd = os.getcwd()
# lst = os.listdir('/root/models')
lf = filter(lambda k: 'model.ckpt-' in k, cwd)
last_model = sorted(lf)[-1].replace('.meta', '')

IndexError: ignored

In [81]:
import os
cwd = os.getcwd()
!python object_detection/export_inference_graph.py \
  --alsologtostderr \
  --model_name=ssd_mobilenet_v1_coco_2018_01_28 \
  --output_file=rahul_graphdef.pb \
  --pipeline_config_path=pipeline.config \
  --trained_checkpoint_prefix=model.ckpt \
  --output_directory= hero

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please switch to tf.train.get_or_create_global_step
Instructions for updating:
Use `tf.profiler.profile(graph, run_meta, op_log, cmd, options)`. Build `options` with `tf.profiler.ProfileOptionBuilder`. See README.md for details
Instructions for updating:
Use tf.compat.v1.graph_util.remove_training_nodes
378 ops no flops stats due to incomplete shapes.
Parsing Inputs...
Incomplete shape.

=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              0
-min_occurrence             0
-step                       -1
-order_by                   name
-a

Check that outputted a file:

In [88]:
!ls -lh rahul*

ls: cannot access 'rahul*': No such file or directory


#### If you only have code creating the graph

Basically you load the graph into memory, run 

```python
from tensorflow.python.framework import graph_util
graph_util.convert_variables_to_constants(...)
```
and save the graph def to disk yourself

### Getting the checkpoint file

When you train your own models, you will likely know where you save checkpoints already.
When you download weights from the internet, they will usually come as a checkpoint.

### Once you have both the **graph definition** file and the **checkpoint** file

Now we just need to combine the structure of the graph from the `.GraphDef` file with the trained weights from a checkpoint. We will write the combined graph with weights into a new `.GraphDef.modelzoo` file:

First we need to get the checkpoint containing gthe trained weights. For slim models, we can download official weights from Google [listed in this readme](https://github.com/tensorflow/models/blob/master/research/slim/nets/nets_factory.py)

In [0]:
!curl -s "https://storage.googleapis.com/download.tensorflow.org/models/nasnet-a_mobile_04_10_2017.tar.gz" | tar xvz

./
./model.ckpt.index
./model.ckpt.data-00000-of-00001


Check that download succeeded:

In [89]:
!ls -lh model*

-rw-r--r-- 1 root root 2.9M May 28 15:22 model.ckpt
-rw-r--r-- 1 root root  27M May 28 15:24 model.ckpt.data-00000-of-00001
-rw-r--r-- 1 root root 8.8K May 28 15:24 model.ckpt.index
-rw-r--r-- 1 root root 2.5M May 28 15:24 model.ckpt.meta


Cool! Now, the freezing script we want to use comes with tensorflow, so we need to find out where our current tensorflow installation lives:

In [90]:
import tensorflow as tf
tf.__file__

'/usr/local/lib/python3.6/dist-packages/tensorflow/__init__.py'

Knowing that path, we can take a look at the `tensorflow/python/tools` subfolder to check if the script is there:

In [91]:
!ls /usr/local/lib/python3.6/dist-packages/tensorflow/python/tools

api			       print_selective_registration_header.py
component_api_helper.py        __pycache__
freeze_graph.py		       saved_model_cli.py
import_pb_to_tensorboard.py    saved_model_utils.py
__init__.py		       selective_registration_header_lib.py
inspect_checkpoint.py	       strip_unused_lib.py
optimize_for_inference_lib.py  strip_unused.py
optimize_for_inference.py


Indeed it is! Let's use it to finally combine the checkpoint and graph definition file into our 'frozen' graph definition format.
The `freeze_graph.py` script takes a couple parameters that are worth talking about:

The first couple are self explanatory and specify input and output files:
```
--input_graph=input_graphdef.pb
--input_checkpoint=model.ckpt
--output_graph=output_graphdef.pb.modelzoo
```

A harder one to pin down is `output_node_names`. Here the script asks you about the name of the output tensor of your model. Usually it will be called something like 'softmax', 'prediction', 'output', etc, and may be nested into scopes, e.g. 'model_name/final_layer/predictions'.
If you don't know that name, take a look at all the nodes in the model graph and see if the naming makes more sense in context. Unfortunately this is up to the model designer and so there are no real standards.



In [93]:
import tensorflow as tf

graph_file = "frozen_inference_graph.pb"
graph_def = tf.GraphDef()
with open(graph_file, "rb") as f:
  graph_def.ParseFromString(f.read())
for node in graph_def.node:
  print(node.name)

image_tensor
ToFloat
Preprocessor/map/Shape
Preprocessor/map/strided_slice/stack
Preprocessor/map/strided_slice/stack_1
Preprocessor/map/strided_slice/stack_2
Preprocessor/map/strided_slice
Preprocessor/map/TensorArray
Preprocessor/map/TensorArrayUnstack/Shape
Preprocessor/map/TensorArrayUnstack/strided_slice/stack
Preprocessor/map/TensorArrayUnstack/strided_slice/stack_1
Preprocessor/map/TensorArrayUnstack/strided_slice/stack_2
Preprocessor/map/TensorArrayUnstack/strided_slice
Preprocessor/map/TensorArrayUnstack/range/start
Preprocessor/map/TensorArrayUnstack/range/delta
Preprocessor/map/TensorArrayUnstack/range
Preprocessor/map/TensorArrayUnstack/TensorArrayScatter/TensorArrayScatterV3
Preprocessor/map/Const
Preprocessor/map/TensorArray_1
Preprocessor/map/TensorArray_2
Preprocessor/map/while/iteration_counter
Preprocessor/map/while/Enter
Preprocessor/map/while/Enter_1
Preprocessor/map/while/Enter_2
Preprocessor/map/while/Enter_3
Preprocessor/map/while/Merge
Preprocessor/map/while/Mer

When you found the output tensor that you're interested in, run the conversion script.

For example, for nasnet_mobile I found the output tensor's name to be 'final_layer/predictions'.

In [131]:
!python /usr/local/lib/python3.6/dist-packages/tensorflow/python/tools/freeze_graph.py \
  --input_graph=frozen_inference_graph.pb \
  --input_checkpoint=model.ckpt \
  --input_binary=true \
  --output_graph=ff.pb.modelzoo \
  --output_node_names=None

Instructions for updating:
Use tf.gfile.GFile.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
2019-05-28 16:46:13.067722: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-05-28 16:46:13.067946: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x2e4dce0 executing computations on platform Host. Devices:
2019-05-28 16:46:13.067985: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
No variables were found in this model. It is likely the model was frozen previously. You cannot freeze a graph twice.


## Specifying modelzoo metadata

In the modelzoo description we specify properties of the model that unfortunately don't usually get encoded into the graph. An example is the pre-processing done to images during training. For exmaple, some models scale an images RGB values from unsigned integer [0, 255] onto something numerically easier to deal with, such as float32 [0.0, 1.0] or float32 [-1.0, 1.0]

First we get lucid to be able to access the modelzoo definitions:

In [0]:
!pip install --quiet lucid

We want to specify our own model, so let's import the `Model` baseclass to inherit from:
TODO: explain or document what options do

In [0]:
from lucid.modelzoo.vision_base import Model

In [0]:
class NasNetMobile(Model):
  model_path = 'frozen_inference_graph.pb'
  image_shape = [254, 254, 3]
  image_value_range = (0, 1)
  input_name = 'input'

Now we can instantiate that new class and use it like normal in lucid:

In [0]:
nasnet = NasNetMobile()
nasnet.load_graphdef()

You need to know which layers, channels or neurons you want to visualize—inspect the model somehow, whether you read the source code or just browse the included tensor names like you did when finding `output_node_names` before:

In [126]:
for node in nasnet.graph_def.node:
  if 'Concat' in node.op:
    print(node.name)

MultipleGridAnchorGenerator/Meshgrid/ExpandedShape/concat
MultipleGridAnchorGenerator/Meshgrid/ExpandedShape_1/concat
MultipleGridAnchorGenerator/Meshgrid_1/ExpandedShape/concat
MultipleGridAnchorGenerator/Meshgrid_1/ExpandedShape_1/concat
MultipleGridAnchorGenerator/Meshgrid_2/ExpandedShape/concat
MultipleGridAnchorGenerator/Meshgrid_2/ExpandedShape_1/concat
MultipleGridAnchorGenerator/concat
MultipleGridAnchorGenerator/Meshgrid_3/ExpandedShape/concat
MultipleGridAnchorGenerator/Meshgrid_3/ExpandedShape_1/concat
MultipleGridAnchorGenerator/Meshgrid_4/ExpandedShape/concat
MultipleGridAnchorGenerator/Meshgrid_4/ExpandedShape_1/concat
MultipleGridAnchorGenerator/Meshgrid_5/ExpandedShape/concat
MultipleGridAnchorGenerator/Meshgrid_5/ExpandedShape_1/concat
MultipleGridAnchorGenerator/concat_1
MultipleGridAnchorGenerator/Meshgrid_6/ExpandedShape/concat
MultipleGridAnchorGenerator/Meshgrid_6/ExpandedShape_1/concat
MultipleGridAnchorGenerator/Meshgrid_7/ExpandedShape/concat
MultipleGridAnchor

Let's take a look at the first channel from a random layer somewhere in the middle of the model:

In [128]:
import lucid.optvis.render as render
_ = render.render_vis(nasnet, "concat_1/concat:0")

ValueError: ignored

TODO: Talk about pre/post relu here!

One isse you will run into is that certain neurons you want to inspect won't activate from just the random noise you feed them, and if they don't activate then a ReLU non-linearity will block all the gradient, so you won't be able to optimize your input.

There are two ways around this: override non-linearity gradients to the identity op for the first couple optimization steps, or create new tensors in your graph that correspond to the tensors you want to optimize, but before their ReLU non-linearity. 

We don't have either of those options automated so far. Look at lucid's googlenet/inception class to see how we "crawl" the graph and add `_pre_relu` nodes for tensors we're interested in.